#### NCSU GIS 714: Geospatial Computation and Simulations

## Surface Water Simulation: Introduction to Surface Water Simulation

Contents:
1. Compare flow accumulation derived by different algorithms
2. Create a map of flooded area
3. Estimate inundation extent using HAND methodology

Additional Resources:
    
* [
GRASS GIS overview and manual](https://grass.osgeo.org/grass80/manuals/index.html)
    
* [GRASS book](http://www.grassbook.org/)
* For more practice in watershed analysis see
[Flow and watershed analysis in GIS582](http://ncsu-geoforall-lab.github.io/geospatial-modeling-course/grass/hydrology.html)

Tip: While executing this notebook, you can launch the GUI in the command line interface with `g.gui`. As you create rasters here, they will appear in the GUI where you can render them, view other layers, etc.

#### Start GRASS GIS

In [ ]:
import os
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
# FOR WINDOWS:
# grass_call = "grass80"
# shell_setting = True
# FOR MAC/LINUX
grass_call = "grass"
shell_setting = False

sys.path.append(
    subprocess.check_output([grass_call, "--config", "python_path"], text=True, shell=shell_setting).strip()
)

# Import GRASS packages
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init("../data", "nc_spm_08_grass7", "PERMANENT")

# Make a new mapset for this assignment
gs.run_command("g.mapset", mapset="HW3_water_simulation", location="nc_spm_08_grass7", flags="c")

### 1. Compare flow accumulation derived by different algorithms

First, we'll set region to the high resolution study area and render it.

In [ ]:
# Set region to the high resolution study area
gs.run_command("g.region", region="rural_1m")

# Create GrassRenderer instance
gs.run_command("r.colors", map="elevation", co="elevation")
img = gj.GrassRenderer(use_region=True)
img.d_rast(map="elev_lid792_1m")
img.d_legend(raster="elev_lid792_1m", at=(55, 95, 80, 84), flags="b")
# Display map
img.show()

##### Least Cost Flow Routing with Multiple Flow Direction
Then, we run least cost path flow routing with multiple flow direction (MFD), implemented with `r.watershed` ([GRASS manual for r.watershed](https://grass.osgeo.org/grass80/manuals/r.watershed.html)). We use the "a" flag to prevent negative values from partial watersheds that go over the edge of the DEM or outside the GRASS computation region.

In [ ]:
# Least Cost Flow Routing with MFD
gs.run_command("r.watershed", elevation="elev_lid792_1m", threshold=5000, accumulation="accum_mfd5K", drainage="draindir_5K", basin="basin_mfd5K", flags="a")

##### Least Cost Flow Routing with Single Flow Direction
#### *Question 1*
    
Use `r.watershed` to repeat the least cost path flow routing above but with single flow direction (SFD/D8). (hint: refer to the [GRASS manual for r.watershed](https://grass.osgeo.org/grass80/manuals/r.watershed.html) to find the appropriate flag. To add multiple flags, put all letters in the same quotations; for example, `flags="abc"`.)

In [ ]:
### Your Answer Here

##### Vector-based Routing
An alternative to MFD and SFD least cost flow routing is vector-based routing (D-inf). In GRASS, this is implemented with `r.flow` ([GRASS manual for r.flow](https://grass.osgeo.org/grass80/manuals/r.flow.html)). Here, we use `r.flow` to compute flowlines, flow length and flow accumulation.

In [ ]:
gs.run_command("r.flow", elevation="elev_lid792_1m", flowline="flowlines", flowlength="flowlg_vb", flowaccumulation="flowacc_vb")

##### Comparison
We will compare our results by looking at the streams created by each method. Here, we demonstrate the stream extraction process for the MFD routing. We extract the streams from the flow accumulation raster by setting cells greater than 100 to 1 and all others to `null`. We thin the result to one-cell width and convert to a vector so that the streams are represented by lines.

In [ ]:
# Extract Streams
gs.mapcalc("{y} = if(abs({x}) > 100, 1, null())".format(y="streams_mfd", x="accum_mfd5K"))

# Thin to one-cell width
gs.run_command("r.thin", input="streams_mfd", output="streams_mfd_t")

# Convert to vector
gs.run_command("r.to.vect", input="streams_mfd_t", out="streams_mfd_t", type="line", flags="s")

#### *Question 2*

Repeat stream extraction process for SFD and vector-based flow routing.

In [ ]:
### Your Answer Here

#### *Question 3*

Use `grass.jupyter.InteractiveMap` to visually compare the resulting stream paths and the flow accumulations, including the official Wake County streams found in `PERMANENT` (`streams@PERMANENT`).

In [ ]:
### Your Answer Here

### 2. Create a map of flooded area
We create a map of flooded area with `r.lake` ([GRASS manual for r.lake](https://grass.osgeo.org/grass80/manuals/r.lake.html)) by providing a water level and a seed point:

In [ ]:
gs.run_command("r.lake", elevation="elev_lid792_1m", water_level=113.5, lake="flood1", coordinates="638728,220278")

# See results
img = gj.GrassRenderer(use_region=True)
img.d_rast(map="elev_lid792_1m")
img.d_rast(map="flood1")
# Display map
img.show()

#### *Question 4*

Increase water level to 113.7m and 114.0m and create flooded area maps at these two levels.

In [ ]:
#### Your Answer Here

### 3. Estimating inundation extent using HAND methodology

We will use two GRASS addons, `r.stream.distance` and `r.lake.series`, to estimate inundation with Height Above Nearest Drainage methodology (A.D. Nobre, 2011). We need to install them first:

In [ ]:
!g.extension r.stream.distance
!g.extension r.lake.series

For this section, we will change our computation region to `elevation` which is a larger study area than we used above. Because we are using a new region (and also a higher threshold of 100,000), we need to run `r.watershed` again to compute the flow accumulation, drainage and streams. We convert the streams to vector for better visualization.

In [ ]:
gs.run_command("g.region", raster="elevation")
gs.run_command("r.watershed", elevation="elevation", accumulation="flowacc", drainage="drainage", stream="streams_100k", threshold=100000)
gs.run_command("r.to.vect", input="streams_100k", output="streams_100k", type="line")

Now we use `r.stream.distance` with output parameter difference to compute new raster where each cell is the elevation difference between the cell and the the cell on the stream where the cell drains. This is our HAND terrain model.

In [ ]:
gs.run_command("r.stream.distance", stream_rast="streams_100k", direction="drainage", elevation="elevation", method="downstream", difference="above_stream")

Before we compute the inundation, we will look at how `r.lake` works. We compute a lake from specified coordinate and water level:

In [ ]:
gs.run_command("r.lake", elevation="elevation", water_level=90, lake="lake", coordinates="637877,218475")

# See results
img = gj.GrassRenderer(use_region=True)
img.d_rast(map="elevation")
img.d_rast(map="lake")
img.d_legend(raster="elevation", at=(55, 95, 80, 84), flags="b")
# Display map
img.show()

Now instead of elevation raster we use the HAND raster to simulate 5-meter inundation and, as the seed, we specify the entire stream instead of a single point.

In [ ]:
gs.run_command("r.lake", elevation="above_stream", water_level=5, lake="flood", seed="streams_100k")

With `r.lake.series`, we can create a series of inundation maps with rising water levels. `r.lake.series` creates a space-time dataset. We can use temporal modules to further work with the data. For example, we could further compute the volume and extent of flood water using `t.rast.univar`.

In [ ]:
gs.run_command("r.lake.series", elevation="above_stream", start_water_level=0, end_water_level=5, water_level_step=0.5, 
               output="inundation", seed_raster="streams")

print(gs.read_command("t.rast.univar", input="inundation", separator="comma"))

#### *Question 5*: Visualize Results
Choose one the following visualization options:

##### Option 1: Create a GIF
Using code from the fractal surfaces section of Assignment 2A part 2, create a GIF showing a timelapse of the inundation.

In [ ]:
### Your Code Here

##### Option 2: Create an InteractiveMap
Use `grass.jupyter.InteractiveMap` to visually compare each step in the inundation space-time dataset.

In [ ]:
### Your Code Here

##### Option 3: Create an animation in MapBox GL
Run the cells below then follow the [grass-mapbox-tutorial](https://github.com/ncsu-geoforall-lab/grass-mapbox-tutorial).

Here, we export each raster as a png and get the bounding box of the computational region in WGS84. We'll do this with the `r.out.leaflet.py` script from the grass-web-publishing GitHub repo. First, clone the plugin using git.

In [ ]:
!git clone https://github.com/ncsu-geoforall-lab/grass-web-publishing.git

Now, we'll make a new directory to store our png images and bounding box file. Then, we run the r.out.leaflet.py python script.

In [ ]:
!mkdir images
!python3 grass-web-publishing/r.out.leaflet/r.out.leaflet.py raster="inundation_0.0,inundation_0.5,inundation_1.0,inundation_1.5,inundation_2.0,inundation_2.5,inundation_3.0,inundation_3.5,inundation_4.0,inundation_4.5,inundation_5.0" output="./images"

You will now see a few new files in the images directory

* data_file.csv    
*  data_file.js
* New pngs

If you are running this through Binder, download the images folder so you have it for the next section.

To add the newly output pngs to a web map continue by following the instructions found in the next tutorial here:

[https://github.com/ncsu-geoforall-lab/grass-mapbox-tutorial](https://github.com/ncsu-geoforall-lab/grass-mapbox-tutorial)

### Additional Information
* [Website](https://github.com/ncsu-geoforall-lab/geospatial-simulations-course)
    
* [Computing Help](https://help.ncsu.edu/)
    
* [GIST Home](https://geospatial.ncsu.edu/)
    
* [Disclaimer](https://www.ncsu.edu/policies/prr-disclaimer.php)
    
* [Accessibility](https://oit.ncsu.edu/itaccess)

* License: 2018 [CC BY-SA](https://creativecommons.org/licenses/by-sa/4.0/)
  
* [NCSU GeoForAll Lab](https://geospatial.ncsu.edu/geoforall/)